In [2]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [3]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
import re
#removing the the aditional and unnecesary text and tags
def clean_text(text):
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [9]:
df["review"] = df["review"].apply(clean_text)

In [ ]:
def lowerCase(text):
    return text.lower() #i coverting the text to lowercase

In [12]:
df["review"] = df["review"].apply(lowerCase)


In [20]:
!python -m spacy download en_core_web_sm



Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 8.9 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 10.6 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 11.0 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 10.8 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.8 MB 10.7 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 10.4 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# def tokenize(text): #only for the token
#     doc = nlp(text)   not optimal
#     return [token.text for token in doc]
def tokenize(text):# token + lemmatization 
    doc = nlp(text)
    token = [
        token.lemma_
        
        for token in doc
        if token.is_alpha
        and (not token.is_stop or token.text in ["not", "no", "never"])
    ]
    return token

# add token column but very slow
df["token"] = df["review"].apply(tokenize)

In [57]:
#test case
text = "The movie was not really fantastic!!!"
print(tokenize(text))

['movie', 'not', 'fantastic']


In [59]:
df.head()


,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [60]:
# fast but not very efficient but here it throughs the error
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer


In [63]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words("english")) - {"not", "no", "never"}
stemmer = PorterStemmer()
tokenizer = TreebankWordTokenizer()

def tokenize2(text):
    tokens = tokenizer.tokenize(text.lower())
    tokens = [stemmer.stem(w) for w in tokens if w.isalpha()]
    tokens = [w for w in tokens if w not in stop_words]
    return tokens




In [ ]:
#drop duplicate
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.duplicated().sum()

np.int64(0)

In [70]:
df["tokens"] = df["review"].apply(tokenize2)

In [71]:
df.head()

,review,sentiment,tokens
0,one of the other reviewers has mentioned that ...,positive,"[one, review, ha, mention, watch, oz, episod, ..."
1,a wonderful little production. the filming tec...,positive,"[wonder, littl, film, techniqu, veri, veri, fa..."
2,i thought this was a wonderful way to spend ti...,positive,"[thought, thi, wa, wonder, way, spend, time, h..."
3,basically there's a family where a little boy ...,negative,"[basic, famili, littl, boy, jake, think, zombi..."
4,"petter mattei's ""love in the time of money"" is...",positive,"[petter, mattei, love, time, money, visual, st..."


In [77]:
df.to_csv("cleaned_data.csv", index=False)
df.head()

,review,sentiment,tokens
0,one of the other reviewers has mentioned that ...,positive,"[one, review, ha, mention, watch, oz, episod, ..."
1,a wonderful little production. the filming tec...,positive,"[wonder, littl, film, techniqu, veri, veri, fa..."
2,i thought this was a wonderful way to spend ti...,positive,"[thought, thi, wa, wonder, way, spend, time, h..."
3,basically there's a family where a little boy ...,negative,"[basic, famili, littl, boy, jake, think, zombi..."
4,"petter mattei's ""love in the time of money"" is...",positive,"[petter, mattei, love, time, money, visual, st..."


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.pipeline import Pipeline

In [87]:
#convert text to vector
vectroizer = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=5000,
    stop_words="english"
)

X = vectroizer.fit_transform(df["review"])
y = df["sentiment"]

In [93]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [96]:
model = LogisticRegression(max_iter=1000, solver="saga")
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'saga'
,max_iter,1000
,multi_class,'deprecated'


In [99]:
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8847433699707573


In [102]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.90      0.87      0.88      4958
    positive       0.87      0.90      0.89      4959

    accuracy                           0.88      9917
   macro avg       0.89      0.88      0.88      9917
weighted avg       0.89      0.88      0.88      9917



In [103]:
#with pipeline


pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=10000, stop_words="english")),
    ("clf", LogisticRegression(max_iter=1000, solver="saga"))
])

X_train, X_test, y_train, y_test = train_test_split(df["review"], df["sentiment"], test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


In [105]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8910960976101644


In [106]:
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(pipeline, f)
    

2.20.0
